# Wordle Game Solver

This project provides an automatic solver for the game Wordle. It takes user feedback about each guess (character and position feedback) and progressively narrows down the set of possible words until it finds the correct word.

## Key Functions:

- `reverse_filter(words, guess)`: Filters the set of possible words based on the user's feedback for each character of a guess.
- `recommend(words)`: Recommends a word to guess. It prioritizes words with unique characters for the first three guesses.
- `get_guess()`: Prompts the user to enter their guess and the corresponding feedback.
- `keep_playing(choice)`: Checks if the user wants to continue playing.

The solver continues refining its list of possible words and making recommendations until the user chooses to stop playing or the correct word is found.

The initial list of words is generated from a large English dictionary and filters down to only include 5-letter words. This could be further optimized by using a dictionary subset specific to the game Wordle, if available.

While the current approach isn't perfectly optimized, it still provides a powerful tool for automating the game of Wordle and showcasing the potential of artificial intelligence in solving word-based puzzles.


In [15]:
import random
from english_words import get_english_words_set
all_words = get_english_words_set(['web2'], lower=True)

In [16]:
import random
from english_words import get_english_words_set  # Importing the required libraries and modules

# Getting the set of all English words
all_words = get_english_words_set(['web2'], lower=True)

# From all the words, we are filtering out the words of length 5
words = [word for word in all_words if len(word) == 5]

# Selecting one random word from the list of 5 letter English words to be the answer
answer = random.choice(words)

def is_good_guess(word):
    """Method returns True if word has length 5 and
    exists in the English dictionary.
    
    Parameters:
    word (str): The word to be checked.

    Returns:
    bool: Returns True if the length of word is 5 and it exists in the words list, False otherwise.
    """
    return len(word) == 5 and word in words

guess = "hello"  # Setting the initial guess

def check_word(guess):
    """Compares the guessed word with the answer for each character.
    grey=no match
    yellow=match but wrong position
    green=correct position
    
    Parameters:
    guess (str): The word guessed by the player.

    Returns:
    str: Returns the score of the guess if it's not correct or 'bingo!' if it's correct.
    """
    # Check if the guessed word is the answer
    if guess == answer:
        return "bingo!"

    # Initialize score for guess
    guess_score = 0

    # For each character in the guess, check if it matches with the corresponding character in the answer
    for i in range(len(guess)):
        if guess[i] == answer[i]:  # The character is in the correct position
            guess_score += 2
            print(f"{i} green")
        elif guess[i] in answer:  # The character is in the word but not in the correct position
            guess_score += 1
            print(f"{i} yellow")
        else:  # The character is not in the word
            print(f"{i} grey")

    # Return the score of the guess
    return f"guess_score={guess_score}"

# Print the answer and the guess for debugging
print(f"answer={answer}")
print(f"guess={guess}")
check_word(guess)  # Invoke the function to check the guess


answer=ichor
guess=hello
0 yellow
1 grey
2 grey
3 grey
4 yellow


'guess_score=2'

In [17]:
from dataclasses import dataclass
from enum import Enum

class CharacterResult(Enum):
    """
    For each character in a guess word, there's a result from this
    enumeration that describes the character's relation to the target
    word's corresponding character.
    
    Values:
    - CORRECT: The character is correct and is in the correct position (Green in NYT app)
    - IN_WORD_WRONG_PLACE: The character is correct but is in the wrong position (Yellow in NYT app)
    - NOT_IN_WORD: The character doesn't exist in the target word (Dark grey in NYT app)
    """
    CORRECT = 2
    IN_WORD_WRONG_PLACE = 1
    NOT_IN_WORD = 0

@dataclass
class GuessResult:
    """
    Describes the result of a guessed word when compared with a target
    word in a game of Wordle.

    Fields:
    - guess: The string that was supplied as the guess
    - id: Identifier mapped to the target word for keeping track of guesses
    - results: An ordered list of character evaluations for the guess
    """
    guess: str
    id: str
    results: list[CharacterResult]

class GameOverException(Exception):
    """
    Raised when the number of guesses for a target word has been
    exceeded. The NYT app allows 6 guesses in total for a given target word.
    """

def evaluate(guess: str, identifier: str | None = None) -> GuessResult:
    """
    Evaluates the supplied guess string against the hidden target
    word mapped to the identifier. If the identifier is None, then a new target
    word is created, and the guess is evaluated against it.

    Parameters:
    - guess: The word to be evaluated
    - identifier: The target word to be evaluated against. If None, a new target word is created.

    Returns:
    - r: The GuessResult object which contains the evaluation result for the guess
    """
    print(f"----looking for answer={identifier}----")
    r = GuessResult(guess, identifier, [])

    for i in range(len(guess)):
        if guess[i] == identifier[i]:  # Character is in the correct position
            r.results.append(CharacterResult.CORRECT)
        elif guess[i] in identifier:  # Character is in the word but not in the correct position
            r.results.append(CharacterResult.IN_WORD_WRONG_PLACE)
        else:  # Character is not in the word
            r.results.append(CharacterResult.NOT_IN_WORD)
    return r


In [19]:
try:
    def filter_words(words, result):
        """
        Filters the set of words based on feedback for each character.

        This function iterates over each character of the guess. If there is only one word left,
        it returns it as the found answer. If not, it filters the words based on the result of 
        the guess (whether the character is in the correct position, in the wrong position, or 
        not in the word) and prints out the filtered words and their count. 

        Parameters:
        - words: The current list of possible words.
        - result: The GuessResult object containing the result of the last guess.

        Returns:
        - words: The filtered list of words.
        """
        for i in range(len(result.guess)):
            if len(words) == 1:
                print("⭐⭐⭐⭐ found it! ⭐⭐⭐⭐")
                return words

            # If character is in the correct position, keep only words that have the same character in the same position
            if result.results[i] == CharacterResult.CORRECT:
                words = [word for word in words if word[i] == result.guess[i]]
                print(f"'{result.guess[i]}' correct position")
                # print(words)
                # print(f"size={len(words)}")

            # If character is in the word but in the wrong position, keep only words that have the same character in a different position
            elif result.results[i] == CharacterResult.IN_WORD_WRONG_PLACE:
                words = [
                    word
                    for word in words
                    if result.guess[i] in word and result.guess[i] != word[i]
                ]
                print(f"'{result.guess[i]}' wrong position")
                # print(words)
                # print(f"size={len(words)}")

            # If character is not in the word, discard words that have the character
            else:
                words = [word for word in words if result.guess[i] not in word]
                print(f"'{result.guess[i]}' not in word")
                # print(words)
                # print(f"size={len(words)}")
        return words

    # Get all 5-character words
    words = [word for word in all_words if len(word) == 5]
    print(f"words_size={len(words)}")

    # Select a random answer
    answer = random.choice(words)

    guess_count = 0
    while guess != answer:
        guess_count += 1

        # Get all words that have unique characters. Unique characters are determined by checking if the set of characters is the same length as the word.
        unique_words = [word for word in words if len(set(word)) == len(word)]

        # Make a guess. For the first guess, use 'adieu'. For the next two guesses, use a random word from the unique_words list. For all subsequent guesses, use a random word from all words.
        if guess_count == 1:
            guess = "adieu"
        elif unique_words and guess_count <= 3:
            guess = random.choice(unique_words)
        else:
            guess = random.choice(words)

        # Evaluate the guess and filter the words based on the result
        result = evaluate(guess, answer)
        words = filter_words(words, result)
        print(f"size={len(words)}")
        print(f"----------guess{guess_count}={guess}----------\n")

        # If the maximum number of attempts has been reached and the guess is still incorrect, raise an exception
        if guess_count == 6 and guess != answer:
            raise GameOverException

# If a GameOverException is raised, print a message and exit the game
except GameOverException:
    print("Dang, we ran out of attempts! 😢😭")

# To use this with the actual game for cheating, we would need to reverse the logic and have the player input character feedback.


words_size=9979
----looking for answer=bunch----
'a' not in word
'd' not in word
'i' not in word
'e' not in word
'u' wrong position
size=589
----------guess1=adieu----------

----looking for answer=bunch----
'w' not in word
'o' not in word
'u' wrong position
'c' correct position
'h' correct position
size=16
----------guess2=wouch----------

----looking for answer=bunch----
'g' not in word
'u' correct position
'l' not in word
'c' correct position
'h' correct position
size=12
----------guess3=gulch----------

----looking for answer=bunch----
'm' not in word
'u' correct position
'n' correct position
'c' correct position
'h' correct position
size=5
----------guess4=munch----------

----looking for answer=bunch----
'b' correct position
⭐⭐⭐⭐ found it! ⭐⭐⭐⭐
size=1
----------guess5=bunch----------



# Improving the Word Guessing Game Strategy

There are several key aspects of our current word guessing strategy that could potentially be improved. The following discussion points could form the basis for further investigation and optimization:

## Starting Word Choice
- Our current approach uses 'adieu' as the first word in every game. This decision seems arbitrary and could potentially be optimized. Two possible alternative strategies come to mind:
    - We could select the word that minimizes the size of the possible word set after the first guess. This would aim to remove as many non-matching words as possible right at the start.
    - Alternatively, we could use a Monte Carlo simulation to select the word that, on average, leads to finding the correct word in the fewest guesses. This approach would consider longer-term outcomes rather than just the immediate impact of the first guess.

## Distinct Character Word Selection
- Currently, we randomly choose words with distinct characters for the second and third guesses. While this strategy likely improves the efficiency of our guessing, it's unclear whether this is the best possible approach. An exploration of alternative word selection methods could be beneficial.

## Handling of Words with Repeated Characters
- Our current approach treats words with repeated characters (like 'daddy') as less likely and leaves them as a lower priority in the selection process. While this approach may generally work well, there may be situations where it leads us astray. For example, if the answer does have repeated characters, our strategy could unnecessarily prolong the game. It could be worthwhile to explore how we can handle words with repeated characters more effectively.

## Improved Guess Optimization
- Is there a more efficient way to optimize our guesses, especially after we have collected some feedback on our previous guesses? We currently rely on random selection from the remaining word set, but perhaps there's a more systematic approach that could be more effective. For example, could we implement a strategy that removes more words per guess without relying on previous feedback?

The answers to these questions aren't immediately apparent and would require a more in-depth analysis and testing. It would be interesting to explore these potential areas for improvement to see how much they could enhance our game strategy.


In [21]:
# This code is designed to "cheat" a word guessing game
# The user inputs character feedback which helps refine the possible set of words

def reverse_filter(words, guess):
    """
    This function filters the possible set of words based on feedback for each character in a guess.
    The guess is a list of tuple pairs - key-value pairs where key is a character of the guess, 
    and value is a feedback ('g' for green/correct position, 'y' for yellow/wrong position, 'x' for grey/not in the word)
    """
    for i in range(len(guess)):
        # If the possible set of words is empty, the word does not exist in the dictionary
        if len(words) == 0:
            print("🤔 word doesn't exist in this dictionary 🤔")
            return words
        # If the possible set of words contains only one word, we've found the correct word
        if len(words) == 1:
            print("⭐⭐⭐⭐ found it! ⭐⭐⭐⭐")
            return words
        
        # Filtering the set of words based on the user's feedback
        if guess[i][1] == "g":
            # If the character is in the correct position ('g' feedback), keep only the words with this character in the same position
            words = [word for word in words if word[i] == guess[i][0]]
            print(f"'{guess[i][0]}' correct position")
        elif guess[i][1] == "y":
            # If the character is in the word but in the wrong position ('y' feedback), keep only the words with this character in a different position
            words = [
                word for word in words if guess[i][0] in word and guess[i][0] != word[i]
            ]
            print(f"'{guess[i][0]}' wrong position")
        else:
            # If the character is not in the word ('x' feedback), exclude the words that contain this character
            words = [word for word in words if guess[i][0] not in word]
            print(f"'{guess[i][0]}' not in word")
    return words

# Building the list of 5-character words
words = [word for word in all_words if len(word) == 5]

def recommend(words):
    """
    This function recommends a word to guess. It prioritizes words with unique characters for the first three guesses.
    """
    # Unique words are the words that have only distinct characters
    unique_words = [word for word in words if len(set(word)) == len(word)]

    # Select the word to guess
    if guess_count == 1:
        guess = "adieu"
    elif unique_words and guess_count <= 3:
        guess = random.choice(unique_words)
    else:
        guess = random.choice(words)
    
    print(f"\nrecommended_guess={guess}")
    return guess

def get_guess():
    """
    This function prompts the user to enter their guess and the corresponding feedback.
    """
    # The user manually selects a guess
    # guess = ""
    # while not is_good_guess(guess):
    #     recommend(words)
    #     guess = input("enter the word you guessed: ")
    #     guess = guess.strip().lower()

    # The function automatically recommends a guess
    guess = recommend(words)

    # Prompt the user to enter the result of their guess
    print(
        f"put the result of '{guess}' as one string like this: gyxxx to say green,yellow,grey,grey,grey"
    )
    result = ""
    while len(result) != 5:
        result = input("enter the result of your guess: ")
        result = result.strip().lower()
        if len(result) != 5:
            print("bad result input")
    # Convert the guess and the result to a list of tuples
    return list(zip(guess, result))

def keep_playing(choice):
    """
    This function checks if the user wants to continue playing.
    """
    return True if 'y' in choice else False

# Initialize the guess counter and user choice
guess_count = 1
choice = "yes"

# Keep playing as long as the user wants to continue
while keep_playing(choice):
    # Filter the possible words based on the user's guess and feedback
    words = reverse_filter(words, get_guess())
    guess_count += 1
    print(f"words_size={len(words)}")
    # If the number of possible words is small, print them out
    if len(words) <= 44:
        print(f"possible_words={words}")
    # If there's only one possible word left, we've found the answer
    if len(words) <= 1:
        print("🎉HA! got em'! 🎉✌")
        break
    # Prompt the user if they want to continue playing
    choice = input("continue? yes or no: ")



recommended_guess=adieu
put the result of 'adieu' as one string like this: gyxxx to say green,yellow,grey,grey,grey
'a' correct position
'd' wrong position
'i' not in word
'e' not in word
'u' not in word
words_size=17
possible_words=['acold', 'alody', 'ayond', 'abdal', 'abody', 'akkad', 'awald', 'ardor', 'arado', 'abdat', 'aldol', 'abord', 'apoda', 'aland', 'alkyd', 'award', 'alada']

recommended_guess=alkyd
put the result of 'alkyd' as one string like this: gyxxx to say green,yellow,grey,grey,grey
'a' correct position
'l' not in word
'k' not in word
'y' not in word
'd' correct position
words_size=2
possible_words=['abord', 'award']

recommended_guess=abord
put the result of 'abord' as one string like this: gyxxx to say green,yellow,grey,grey,grey
'a' correct position
'b' not in word
⭐⭐⭐⭐ found it! ⭐⭐⭐⭐
words_size=1
possible_words=['award']
🎉HA! got em'! 🎉✌
